# Part 2 of interpretability of a Convnet model


In this case the idea now is to plot what the filters see when they activate at their highest level

In [ ]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import math

# load the 3 models we already've trained

In [ ]:
model1 = tf.keras.models.load_model("./checkpoints/model1/")

model2 = tf.keras.models.load_model("./checkpoints/model2/")

model3 = tf.keras.models.load_model("./checkpoints/esp32/")

# pretrained models
# model_pretrained = tf.keras.applications.InceptionV3(include_top=False)
model_pretrained = tf.keras.applications.ResNet50(include_top=False)

# First step build a loss function

The idea is to maximize the activation

In [ ]:
test_tensor = tf.expand_dims(tf.random.uniform(minval=0.4, maxval=0.6, shape=(256, 256, 3)), axis=0)

In [ ]:
tf.reduce_mean(test_tensor)

In [ ]:
tf.reduce_mean(tf.math.rsqrt(tf.math.reduce_euclidean_norm(test_tensor, axis=2)))

In [ ]:
tf.math.reduce_euclidean_norm(tf.reduce_mean(test_tensor, axis=2))

In [ ]:
def loss(activation):
    """
        initial_random_image: initial random image made to max out the activation of the filter.
        prediction: output of the filter's activation.
        Squared Error

        # Trying to maximize the activations average.
    """

    return tf.reduce_mean(activation)

In [ ]:
def preproces_img(image_output):

    filter_patern = image_output.copy()

    mean = filter_patern.mean()
    std = filter_patern.std()

    filter_patern -= mean
    filter_patern /= std

    filter_patern *= 64
    filter_patern += 128
    
    filter_patern = np.clip(filter_patern, 0, 255).astype("uint8")

    #Center image
    return filter_patern[25:-25, 25:-25, :]

In [ ]:
def get_convs_layers(model):
    outputs=[]
    for layer in model.layers:
        if isinstance(layer, (tf.keras.layers.Conv2D, tf.keras.layers.MaxPool2D)):
            layer.trainable=False
            outputs.append(layer)
    return outputs

In [ ]:
def print_filters(model_name, layer_name, all_images, image_shape=(256, 256)):       
    margin = 5
    c = 4
    r = math.ceil(len(all_images)/c)

    cropped_width = image_shape[0]-25*2
    cropped_height = image_shape[1]-25*2

    width = r * cropped_width + (r-1) * margin
    height = c * cropped_height + (c-1) * margin

    stitched_filters = np.zeros((width, height, 3))

    #rows
    for i in tqdm(range(r)):
        for j in range(c):
            image = all_images[i * c +j]

            row_start= (cropped_width + margin) * i
            row_end = (cropped_height + margin) * i + cropped_width
            column_start = (cropped_height + margin) * j
            column_end = (cropped_height + margin) * j + cropped_height

            stitched_filters[row_start: row_end, column_start: column_end, :] = image

    tf.keras.utils.save_img(f"./part2-filters/{model_name}/{layer_name}.png", stitched_filters)

In [ ]:
@tf.function
def optimize_step(feature_extractor_model, image, filter_index, lr):
    with tf.GradientTape() as tape:
        tape.watch(image)

        activation = feature_extractor_model(image)
        activation = activation[:, 2:-2, 2:-2, filter_index]

        loss_value = loss(activation)

    gradients = tape.gradient(loss_value, image)
    gradients = tf.math.l2_normalize(gradients)

    image += gradients*lr

    return image, loss_value

In [ ]:
model3.summary()

In [ ]:
model = model3

filter_index = 0
sample_layer = model.get_layer("2nd_conv_3x3")
feature_extractor_model = tf.keras.Model(inputs=model.input, outputs=sample_layer.output)

lr=10.

In [ ]:
sample_layer.weights

In [ ]:
sample_layer.output

In [ ]:
kernel = sample_layer.weights[0][:, :, 0, 7]

kernel *= 64
kernel += 128

kernel = np.array(np.clip(kernel, a_min=0, a_max=255), dtype=np.uint8)

print(kernel)

plt.imshow(kernel)
plt.show()

In [ ]:
import os
import shutil
import gc

tf.keras.backend.clear_session()

image_shape_list = [(256, 256, 3), (256, 256, 3), (96, 96, 1), (299, 299, 3)]

models=[model1, model2, model3, model_pretrained]
models_name = ["model_1_new_version", "model_2_new_version", "model_3_new_version", "model_resnet50"]

epochs = 500
learning_rate = 10.

for model_name, model, image_shape in zip(models_name, models, image_shape_list):
    print(f"Model : {model_name}")

    if not os.path.exists(f"part2-filters/{model_name}/"):
        os.mkdir(f"part2-filters/{model_name}/")
    else:
        shutil.rmtree(f"part2-filters/{model_name}/")
        os.mkdir(f"part2-filters/{model_name}/")

    for layer in get_convs_layers(model):
        feature_extractor_model = tf.keras.Model(inputs=model.input, outputs=layer.output)
        images = []

        filters_count = layer.output.shape[-1]

        for filter_index in range(filters_count):
            # noisy_image = tf.expand_dims(tf.random.uniform(minval=0.4, maxval=0.6, shape=image_shape), axis=0)
            noisy_image = tf.expand_dims(tf.random.normal(shape=image_shape), axis=0)

            for epoch in range(epochs):
                noisy_image, loss_value = optimize_step(feature_extractor_model, noisy_image, filter_index, learning_rate)

            images.append(preproces_img(noisy_image.numpy()[0].copy()))
        
        del feature_extractor_model
        tf.keras.backend.clear_session()
        gc.collect()

        print_filters(model_name=model_name, layer_name=layer.name, all_images=images, image_shape=image_shape)